<h1><center>UCZENIE MASZYNOWE W ANIMACJACH</center></h1>

<br></br>

<center><img src="pictures/logo.png"/></center>
<br></br>

<h2><center>Prowadzący: mgr inż. Szymon Datko</center></h2>

<h3>Grupa Alfa</h3>
<ul>
<li>Adam Polerowicz 243442</li>
<li>Marcin Wawszczak</li>
<li>Bartosz Olszewski</li>
</ul>

<br></br>

<h2>Temat: Wykorzystanie sieci GAN do zadania upscalingu</h2>

<br></br>

<h4>Zadanie polegało na wykorzystaniu sieci GAN do zwiększania rozdzielczości obrazów.</h4>

#  Implementacje open-source

<br></br>

Sieci GAN są stosunkowo nową architekturą. Szybko jednak znalazły zastosowanie przy zadaniach upscalingu i augmentacji. Na początku należało przeprowadzić analize dostępnych rozwiązań. Powszechnie znanym repozytorium wszelkich open-sourcowych projektów jest Github. Pośród wielu implementacji sieci GAN do zwiększania rozdzielczości obrazów [1][2], na szczególną uwagę zasługuje repozytorium IBM [3]. Zawiera ono miedzy innymi kod do szybkiej implementacji upscalera. Jest on w stanie zwiększyć rozdzielczość obrazu czterokrotnie. Użyty model został wytrenowany na 600 000 obrazach z OpenImagesV4 [4]. Jakość uzyskiwanych wyników została sprawdzona na trzech zbiorach danych (Set4, Set14 oraz BSD100). Użyto metryk PSNR (ang. peak signal to noise ratio) oraz SSIM (structural similarity index). Repozytorium zawiera API, które ułatwia pracę z modelem. Jego uruchomienie sprowadza się do ściągnięcia obrazu z rejestru Quay.io [5] poprzec Dockera [6]. 

Przeprowadzono kilkanaście inferencji z modelem używając obrazów znalezionych w grafice Google. Jakość uzyskanych obrazów oceniona organoleptycznie została określona jako nieidealna i w zależności od użytego obrazu pojawiały się artefakty.

<br></br>
<h4> Przykłady obrazów mocno odbiegających od oryginału </h4>
<br></br>

![10](.\pictures\Enchanced\10en.png)
![3](.\pictures\Enchanced\3en.png)

<br></br>
<h4> Przykłady obrazów prawie idealnych (drobne artefakty) </h4>
<br></br>

![6](.\pictures\Enchanced\6en.png)
![4](.\pictures\Enchanced\4en.png)

<br></br>
<h4> Przykłady obrazów nie do odróżnienia z oryginalnymi </h4>
<br></br>

![5](.\pictures\Enchanced\5en.png)
![7](.\pictures\Enchanced\7en.png)


# Metryki
<br></br>
W celu przeprowadzenia porównania obrazów referencyjnych i po uscalingu na podstawie innej niż organoleptyczna stworzono skrypty do liczenia SSIM oraz MSE (ang. mean square error). Nie można jednak całkowicie wykluczyć bliżej nieokreślonej metryki jaką jest ocena organoleptyczna. Autorzy pracy naukowej [7], na podstawie której został stworzony model IBM sugerują, że MOS (z ang. Mean Opinion Score) jest najlepszą metryką do oceny modelu tego typu. MOS jest średnią opinią tworzoną przez subiektywną opinię ludzi.

In [9]:
from PIL import Image
from pathlib import Path

def resize_images(path : str, extension='png'):
    resize_factor = 4
    base_path = Path(path)

    for image_path in base_path.glob('*.' + extension):
        img = Image.open(image_path)
        size = img.size
        new_size = (int(size[0] / resize_factor), int(size[1] / resize_factor))
        print('Original: {}'.format(size) + ' New: {} \t\t File {}'.format(new_size, image_path.name))

In [11]:
resize_images('./pictures/Originals')

Original: (1920, 1080) New: (480, 270) 		 File 1.png
Original: (1920, 1080) New: (480, 270) 		 File 10.png
Original: (1920, 1080) New: (480, 270) 		 File 2.png
Original: (1920, 1080) New: (480, 270) 		 File 3.png
Original: (1920, 1080) New: (480, 270) 		 File 4.png
Original: (1920, 1080) New: (480, 270) 		 File 5.png
Original: (1920, 1080) New: (480, 270) 		 File 6.png
Original: (1920, 1080) New: (480, 270) 		 File 7.png
Original: (1920, 1080) New: (480, 270) 		 File 8.png
Original: (1920, 1080) New: (480, 270) 		 File 9.png
Original: (1920, 1080) New: (480, 270) 		 File 9_white.png


In [6]:
from skimage.metrics import structural_similarity as ssim
from pandas import *
import numpy as np
import cv2

def mse(imageA, imageB):
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	return err

def print_compare(pathA : str, pathB : str, suffix='', extensions=['png', 'png']):
    base_path = Path(pathA).parent
    names, mses, sims = [], [], [] 
    
    for org_path in Path(pathA).glob('*.' + extensions[0]):
        orig = cv2.imread(str(Path(pathA).joinpath(str(org_path.stem) + '.' + extensions[0])))
        ench = cv2.imread(str(Path(pathB).joinpath(str(org_path.stem) + suffix + '.' +  extensions[1])))
        orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
        ench = cv2.cvtColor(ench, cv2.COLOR_BGR2GRAY)	
        mses.append(float("{:.2f}".format(mse(orig, ench))))
        sims.append(float("{:.4f}".format(ssim(orig, ench))))
        names.append(org_path.stem)

    merge = [names, mses, sims, [element * 100 for element in sims]]
    df = DataFrame(merge)
    col = ['Image name', 'MSE', 'SSIM', 'SSIM %']
    df.insert(0, "Id", col, True)
    print(df.to_string(index=False))
    print('Mean MSE: {:.2f}'.format(np.mean(mses)))
    print('Mean SSIM: {:.2f} %'.format(np.mean(merge[3])))

In [12]:
print_compare('./pictures/Originals/', './pictures/Enchanced/', 'en')

        Id       0       1         2       3       4      5       6       7        8         9      10
Image name       1      10         2       3       4      5       6       7        8         9 9_white
       MSE  835.81  393.05  24262.87  170.67   19.74  80.59   32.44    6.84  3119.12  11698.99  206.67
      SSIM  0.4739  0.8666    0.5329  0.7372  0.9307  0.903  0.9445  0.9838   0.8559    0.2107  0.7717
    SSIM %   47.39   86.66     53.29   73.72   93.07   90.3   94.45   98.38    85.59     21.07   77.17
Mean MSE: 3711.53
Mean SSIM: 74.64 %


# Samodzielne implementacje

<br></br>

Po wykonaniu przeglądu dostępnych rozwiązań podjęto się zadania wytrenowania sieci GAN samodzielnie. Wykorzystano w tym celu PyTorch-ową [8] implementacje architektury SRGAN [9]. Sieci GAN ze względu na swoją strukturę można opisać jako dwie współpracujące sieci. Oznacza to, że wytrenowanie sieci na podobnej ilości obrazów jak referencyjna sieć (600 000) wymagałoby dużo zasobów zarówno sprzętowych jak i czasowych. Postanowiono przeprowadzić porównanie sieci IBM z siecią  wytrenowaną na dużo mniejszym zbiorze ograniczonym do konkretnej tematyki. Wykorzystano zbiór danych URBAN100, który zawiera 100 zdjęć architektury i budowli z całego świata. Aby móc wytrenować sieć na tych danych należało skorzystać ze skryptu dzielącego zbiór na zbiór treningowy i walidacyjny. Strategia tworzenia danych potrzebnych do wytrenowania sieci sprowadza się na podziale pojedyńczego zdjęcia na kilkanaście równych sobie co do wymiarów obrazów. W ten sposób ze stu obrazów postał zbiór treningowy, który liczy 29 020 instancji oraz walidacyjny z 1 451 instancjami. Sieć została wytrenowana na sprzęcie o następujących parametrach:
<ul>
    <li> CPU i7 6500u </li>
    <li> GPU GTX 940mx </li>
</ul>
Trening trwał około 36 godzin i został zatrzymany przy 33 epoce (early stopping).

Po uzyskaniu najlepszego modelu przystąpiono do porównania z siecią IBM. Użyto dzisięciu obrazów, które przedstawiały budowle.

In [13]:
resize_images('./pictures/Original', 'jpg')

Original: (480, 360) New: (120, 90) 		 File 1.jpg
Original: (384, 384) New: (96, 96) 		 File 2.jpg
Original: (264, 400) New: (66, 100) 		 File 3.jpg
Original: (336, 224) New: (84, 56) 		 File 4.jpg
Original: (1024, 892) New: (256, 223) 		 File 5.jpg
Original: (400, 348) New: (100, 87) 		 File 6.jpg
Original: (300, 300) New: (75, 75) 		 File 7.jpg
Original: (300, 300) New: (75, 75) 		 File 8.jpg
Original: (316, 224) New: (79, 56) 		 File 9.jpg


W ten sposób przygotowane obrazy podano sieci IBM i SRGAN jako dane wejściowe. W efekcie otrzymano czterokrotnie większe obrazy. Sieć SRGAN domyślnie zapisuje kanały obrazów w kolejności BGR a nie RGB. W celu ujednolicenia kolejności kolorów przerobiono je na RGB.

In [14]:
import math

def bgr2rgb(path : str, extension='png'):
    test_path = Path(path)

    for path in test_path.glob('*.' + extension):
        img = cv2.imread(str(path))
        img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

Nastepnie porównano wyniki:

In [15]:
print_compare('./pictures/IBM','./pictures/Original', extensions=['png', 'jpg'])

        Id       0       1      2       3       4       5       6        7       8
Image name       1       2      3       4       5       6       7        8       9
       MSE  250.75  224.36  352.0  392.52  488.94  517.36   652.6  1523.54  567.86
      SSIM  0.6504  0.6873  0.722  0.6343  0.6703  0.7356  0.6299   0.4812  0.7047
    SSIM %   65.04   68.73   72.2   63.43   67.03   73.56   62.99    48.12   70.47
Mean MSE: 552.21
Mean SSIM: 65.73 %


In [16]:
print_compare('./pictures/SRGAN','./pictures/Original', extensions=['jpg', 'jpg'])

        Id       0       1       2       3       4       5       6        7       8
Image name       1       2       3       4       5       6       7        8       9
       MSE  251.84  203.31   234.7  355.36  395.97   459.6  449.35  1600.13  475.05
      SSIM  0.6688  0.6945  0.7994  0.6518  0.7014  0.7565  0.7033    0.495  0.7259
    SSIM %   66.88   69.45   79.94   65.18   70.14   75.65   70.33     49.5   72.59
Mean MSE: 491.70
Mean SSIM: 68.85 %


# Porównanie IBM i SRGAN do oryginalnych zdjęć 
<br></br>
Analizując wyniki porównania powiększonych zdjęć do oryginalnych stwierdzono kilkuprocentową przewagę sieci SRGAN. Różnica w metryce MSE wynosi ponad 12%. Oznacza to, że sieć wyszkolona w wąskiej tematyce poradziła sobie lepiej z upscalingiem obrazów nawiązujących do niej. Metryka MOS (użyto trzech osób) potwierdziła subtelną przewagę sieci SRGAN.
<br></br>
Kolejność:
<ol>
    <li>Oryginał</li>
    <li>SRGAN</li>
    <li>GAN</li>
</ol>

<br></br>
![3](.\pictures\Original\3.jpg)
![3](.\pictures\SRGAN\3.jpg)
![3](.\pictures\IBM\3.png)
<br></br>
![9](.\pictures\Original\9.jpg)
![9](.\pictures\SRGAN\9.jpg)
![9](.\pictures\IBM\9.png)
<br></br>
![5](.\pictures\Original\5.jpg)
![5](.\pictures\SRGAN\5.jpg)
![5](.\pictures\IBM\5.png)

# Zmiana czynnika upscalingu

<br></br>

Sieć SRGAN opisywana w powyższych rozdziałach została wytrenowana do upscalingu obrazów o czynnik równy 4. Pozwoliło to na porównanie wytrenowanego modelu z modelem IBM. Podjęto póbę stworzenia sieci, która cechowałaby się czynnikiem upscalingu równym 8 lub 16. 
Skorzystano z gotowej implementacji sieci [10]. W celu wytrenowania modelu użyto zbioru DIV2K [11]. Model wykorzystuje bloki pola receptywnego (RFB), które umożliwiają wydobycie infromacji z obrazów w większej skali i zwiększenie możliwości dyskryminacyjnych modelu. Ponadto, zamiast używać dużych jąder splotowych w wieloskalowym bloku receptywnym, model korzysta z kilku małych jąder, dzięki czemu jest w stanie wyodrębnić większe szczegóły z obrazów i zmniejszyć złożoność obliczeń. Ostatnią włąsnością modelu jest fakt, że naprzemiennie wykorzystuje różne metody upsamplingu, aby zmniejszyć dużą złożoność obliczeń i zachować zadowalającą wydajność.
Przygotowano zbiór danych, dzieląc go na zbiór treningowy i walidacyjny. Z powodzeniem ropoczęto proces trenowania sieci, który początkowo przewidziany na 100 epok, w ciągu 36 godzin nieustannej pracy zakończono (umyślnie) w połowie pierwszej epoki. Złożoność modelu wraz z faktem dużo większych danych uczących (rozmiar zdjęcia) oraz małej mocy obliczeniowej okazała się być powodem niepowodzenia wytrenowania sieci. 

Podjęto próbę wytrenowania innego modelu z czynnikiem upscalingu równym 2. Wykorzystano kod z repozytorium SRGAN [12]. Do treningu ponownie użyto zbioru DIV2K, który przygotowano dzieląc każdy z obrazów na kilka o mniejszej rozdzielczości (z pojedynczego 2560x1440 na wiele 600x600 pikseli). Z powodzeniem uruchomiono trening i z takim samym rezultatem go zakończono. Poniżej przedstawiono uzyskane rezultaty.

# Kod

<br></br>
https://github.com/BAPO-Developers/ImageUpscaler_GAN_project

# Referencje
<br></br>
<ul>
    <li>[1] https://github.com/brade31919/SRGAN-tensorflow</li>
    <li>[2] https://github.com/trevor-m/tensorflow-SRGAN</li>
    <li>[3] https://github.com/IBM/MAX-Image-Resolution-Enhancer</li>
    <li>[4] https://storage.googleapis.com/openimages/web/index.html</li>
    <li>[5] https://quay.io </li>
    <li>[6] https://www.docker.com</li>
    <li>[7] C. Ledig, L. Theis, F. Huszar, J. Caballero, A. Cunningham, A. Acosta, A. Aitken, A. Tejani, J. Totz, Z. Wang, W. Shi, Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network, ArXiv, 2017.</li>
    <li>[8] https://pytorch.org </li>
    <li>[9] https://github.com/Lornatang/SRGAN-PyTorch </li>
    <li>[10] https://github.com/Lornatang/RFB_ESRGAN-PyTorch </li>
    <li>[11] https://data.vision.ee.ethz.ch/cvl/DIV2K/ </li>    
    <li>[12] https://github.com/leftthomas/SRGAN </li>    
    
   
</ul>